In [1]:
!pip install -q -U torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [3]:
!pip install -q -U faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.7 MB/s eta 0:00:00:00:0100:01


In [46]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import numpy as np
import faiss
import requests
import os
from torch.optim import AdamW
from torch.nn.functional import pad

In [6]:
df = pd.read_csv('/kaggle/input/mini-product-image-and-text-dataset/data.csv')
df.head()

,image,description,display name,category
0,3238.jpg,"Round toed, black sports shoes with red accent...",Puma Men Black 65CC Lo Ducati Sports Shoes,Sports Shoes
1,43044.jpg,Style Note Built with the breathability and ze...,Nike Men Charcoal Grey Shorts,Shorts
2,54018.jpg,Teal handbag that has stitch detailing with a...,Kiara Women Teal Handbag,Handbags
3,8141.jpg,"Perfectly stylish, this fastrack analog wrist ...",Fastrack Women Freestyle Sports Analog Steel B...,Watches
4,22245.jpg,These id mid-top chukka shoes add a fresh spin...,ID Men Brown Casual Shoes,Casual Shoes


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14').to(device)
processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [59]:
row = df.iloc[6]

txt = str(row['description']) + str(row['display name']) + str(row['category'])

txt

'Crafted for the urban style maven, this brown wallet from lino perros is a must have for those who love organising their wallet space. Dimensions : 10 cm x 19 cm Made of leather Two insert pockets Multi card slots inside One id card holder with a mesh Six more card slots on the other side with an insert pocket The flap has a clasp button Care Wipe with clean, damp cloth to remove dirtLino Perros Women Leather Brown WalletWallets'

In [45]:
print(processor.tokenizer.model_max_length)

77


In [40]:
class ProductDataset(Dataset):
    def __init__(self, metadata, processor):
        self.metadata = metadata
        self.processor = processor

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image = Image.open(f"/kaggle/input/mini-product-image-and-text-dataset/data/{row['image']}")
        text = str(row['description']) + str(row['display name']) + str(row['category'])
        inputs = self.processor(text=[text], images=image, return_tensors='pt', padding=True, truncation=True)

        return{
            "input_ids" : inputs['input_ids'].squeeze(0),
            "attention_mask" : inputs['attention_mask'].squeeze(0),
            "pixel_values" : inputs['pixel_values'].squeeze(0)
        }

In [41]:
dataset = ProductDataset(df, processor)

In [43]:
sample = dataset[0]
print(sample.keys())

print(sample["input_ids"].shape)
print(sample["attention_mask"].shape)
print(sample["pixel_values"].shape)

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])
torch.Size([77])
torch.Size([77])
torch.Size([3, 224, 224])


In [44]:
print("Total samples:", len(dataset))

Total samples: 44441


In [48]:
print("Pad token id :", processor.tokenizer.pad_token_id)

Pad token id : 49407


In [50]:
def custom_collate_func(batch):
    max_len = max([len(item['input_ids']) for item in batch])
    
    padded_input_ids = torch.stack([
        pad(item['input_ids'], (0, max_len - len(item['input_ids'])), 
        value=processor.tokenizer.pad_token_id) for item in batch
    ])
    
    padded_attention_mask = torch.stack([
        pad(item['attention_mask'], (0, max_len - len(item['attention_mask'])), 
        value=processor.tokenizer.pad_token_id) for item in batch
    ])

    pixel_values = torch.stack([item['pixel_values'] for item in batch])

    return{
        "input_ids" : padded_input_ids,
        "attention_mask" : padded_attention_mask,
        "pixel_values" : pixel_values
    }

In [53]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=custom_collate_func)

In [54]:
len(dataloader)

22221

#### Fine-Tune CLIP